# Apriori Algorithm Analysis
## Minimum Support = 0.3

This notebook analyzes transaction data to find frequent itemsets and generate association rules.

In [9]:
import pandas as pd
from itertools import combinations
from collections import Counter

# Transaction data from the CSV
transactions = [
    ['Bread', 'Milk', 'Eggs'],
    ['Bread', 'Butter', 'Cheese'],
    ['Milk', 'Eggs', 'Butter'],
    ['Bread', 'Milk', 'Butter'],
    ['Bread', 'Eggs', 'Apples'],
    ['Milk', 'Eggs', 'Cheese'],
    ['Bread', 'Milk', 'Eggs', 'Butter'],
    ['Milk', 'Butter', 'Bananas'],
    ['Bread', 'Eggs', 'Butter'],
    ['Milk', 'Eggs', 'Butter', 'Cheese']
]

# Abbreviations for display
abbrev = {
    'Bread': 'B',
    'Milk': 'M',
    'Eggs': 'E',
    'Butter': 'Bu',
    'Cheese': 'C',
    'Apples': 'A',
    'Bananas': 'Ba'
}

total_transactions = len(transactions)
min_support = 0.3
min_count = min_support * total_transactions

print(f"Total Transactions: {total_transactions}")
print(f"Minimum Support: {min_support}")
print(f"Minimum Count: {min_count}")
print(f"\nTransactions:")
for i, trans in enumerate(transactions, 1):
    print(f"{i}. {', '.join(trans)}")

Total Transactions: 10
Minimum Support: 0.3
Minimum Count: 3.0

Transactions:
1. Bread, Milk, Eggs
2. Bread, Butter, Cheese
3. Milk, Eggs, Butter
4. Bread, Milk, Butter
5. Bread, Eggs, Apples
6. Milk, Eggs, Cheese
7. Bread, Milk, Eggs, Butter
8. Milk, Butter, Bananas
9. Bread, Eggs, Butter
10. Milk, Eggs, Butter, Cheese


## Step 1: Find Frequent 1-Itemsets

## Understanding Support

**Support** measures how frequently an itemset appears in the dataset.

### Formula:
```
Support(X) = Count(X) / Total Transactions
```

### Examples with our data (10 transactions):

**1-Itemset:**
- If Milk appears in 7 transactions: Support(Milk) = 7/10 = 0.7

**2-Itemset:**
- If {Bread, Milk} appears in 3 transactions: Support(Bread, Milk) = 3/10 = 0.3

**3-Itemset:**
- If {Bread, Milk, Eggs} appears in 2 transactions: Support(Bread, Milk, Eggs) = 2/10 = 0.2

### Minimum Support Threshold:
- Minimum support = 0.3 means an itemset must appear in at least 3 transactions (0.3 x 10 = 3)
- Any itemset with support < 0.3 is NOT frequent and is pruned

In [10]:
# Count 1-itemsets
item_counts = Counter()
for transaction in transactions:
    for item in transaction:
        item_counts[item] += 1

# Filter by minimum support
frequent_1_itemsets = {item: count for item, count in item_counts.items() if count >= min_count}

# Create DataFrame for display
df_1_itemsets = pd.DataFrame([
    {'Item': abbrev[item], 'Count': count, 'Support': count/total_transactions}
    for item, count in sorted(frequent_1_itemsets.items(), key=lambda x: x[1], reverse=True)
])

print("Frequent 1-Itemsets:")
print(df_1_itemsets.to_string(index=False))
print(f"\nTotal frequent 1-itemsets: {len(frequent_1_itemsets)}")

Frequent 1-Itemsets:
Item  Count  Support
   M      7      0.7
   E      7      0.7
  Bu      7      0.7
   B      6      0.6
   C      3      0.3

Total frequent 1-itemsets: 5


## Step 2: Find Frequent 2-Itemsets

In [11]:
# Count 2-itemsets (only from frequent 1-itemsets)
pair_counts = Counter()
frequent_items = set(frequent_1_itemsets.keys())

for transaction in transactions:
    # Only consider items that are frequent
    frequent_in_trans = [item for item in transaction if item in frequent_items]
    # Generate all 2-item combinations
    for pair in combinations(sorted(frequent_in_trans), 2):
        pair_counts[pair] += 1

# Filter by minimum support
frequent_2_itemsets = {pair: count for pair, count in pair_counts.items() if count >= min_count}

# Create DataFrame for display
df_2_itemsets = pd.DataFrame([
    {
        'Itemset': f"{{{abbrev[pair[0]]}, {abbrev[pair[1]]}}}",
        'Items': f"{pair[0]}, {pair[1]}",
        'Count': count,
        'Support': round(count/total_transactions, 2)
    }
    for pair, count in sorted(frequent_2_itemsets.items(), key=lambda x: x[1], reverse=True)
])

print("\nFrequent 2-Itemsets (Minimum Support = 0.3):")
print(df_2_itemsets.to_string(index=False))
print(f"\nTotal frequent 2-itemsets: {len(frequent_2_itemsets)}")


Frequent 2-Itemsets (Minimum Support = 0.3):
Itemset         Items  Count  Support
 {E, M}    Eggs, Milk      5      0.5
{Bu, M}  Butter, Milk      5      0.5
 {B, E}   Bread, Eggs      4      0.4
{B, Bu} Bread, Butter      4      0.4
{Bu, E}  Butter, Eggs      4      0.4
 {B, M}   Bread, Milk      3      0.3

Total frequent 2-itemsets: 6


## Step 3: Find Frequent 3-Itemsets

In [12]:
# Count 3-itemsets (only from frequent 2-itemsets)
triple_counts = Counter()

for transaction in transactions:
    # Only consider items that are frequent
    frequent_in_trans = [item for item in transaction if item in frequent_items]
    # Generate all 3-item combinations
    for triple in combinations(sorted(frequent_in_trans), 3):
        # Check if all 2-item subsets are frequent (Apriori property)
        subsets_frequent = all(
            tuple(sorted(pair)) in frequent_2_itemsets
            for pair in combinations(triple, 2)
        )
        if subsets_frequent:
            triple_counts[triple] += 1

# Filter by minimum support
frequent_3_itemsets = {triple: count for triple, count in triple_counts.items() if count >= min_count}

# Create DataFrame for display
df_3_itemsets = pd.DataFrame([
    {
        'Itemset': f"{{{abbrev[triple[0]]}, {abbrev[triple[1]]}, {abbrev[triple[2]]}}}",
        'Items': f"{triple[0]}, {triple[1]}, {triple[2]}",
        'Count': count,
        'Support': round(count/total_transactions, 2)
    }
    for triple, count in sorted(frequent_3_itemsets.items(), key=lambda x: x[1], reverse=True)
])

print("\nFrequent 3-Itemsets (Minimum Support = 0.3):")
if len(df_3_itemsets) > 0:
    print(df_3_itemsets.to_string(index=False))
else:
    print("No frequent 3-itemsets found with minimum support of 0.3")
print(f"\nTotal frequent 3-itemsets: {len(frequent_3_itemsets)}")


Frequent 3-Itemsets (Minimum Support = 0.3):
   Itemset              Items  Count  Support
{Bu, E, M} Butter, Eggs, Milk      3      0.3

Total frequent 3-itemsets: 1


## Step 4: Generate Association Rules from 2-Itemsets

### Understanding Confidence

**Confidence** measures how often items in the consequent appear in transactions that contain the antecedent.

### Formula:
```
Confidence(A -> B) = Support(A, B) / Support(A)
                   = Count(A and B together) / Count(A)
```

This tells us: "If a customer buys A, what is the probability they also buy B?"

### Example Calculation:

Consider the rule **Bread -> Milk**:
- Count of {Bread, Milk} together = 3 transactions
- Count of Bread alone = 6 transactions
- Confidence = 3/6 = 0.50 = **50%**

**Interpretation:** When customers buy Bread, they also buy Milk 50% of the time.

### Why Generate Both Directions?

For itemset {A, B}, we generate TWO rules:
1. **A -> B**: Confidence = Count(A,B) / Count(A)
2. **B -> A**: Confidence = Count(A,B) / Count(B)

These usually have DIFFERENT confidence values because:
- A might be common (appears often) -> lower confidence for A -> B
- B might be rare (appears less) -> higher confidence for B -> A

In [13]:
rules_2 = []

for pair, pair_count in frequent_2_itemsets.items():
    item1, item2 = pair
    support_pair = pair_count / total_transactions
    
    # Rule: item1 -> item2
    support_item1 = frequent_1_itemsets[item1] / total_transactions
    confidence_1_to_2 = pair_count / frequent_1_itemsets[item1]
    
    rules_2.append({
        'Rule': f"{abbrev[item1]} -> {abbrev[item2]}",
        'Full Rule': f"{item1} -> {item2}",
        'Support': round(support_pair, 2),
        'Confidence': round(confidence_1_to_2, 2),
        'Calculation': f"{pair_count}/{frequent_1_itemsets[item1]} = {confidence_1_to_2:.2f}"
    })
    
    # Rule: item2 -> item1
    support_item2 = frequent_1_itemsets[item2] / total_transactions
    confidence_2_to_1 = pair_count / frequent_1_itemsets[item2]
    
    rules_2.append({
        'Rule': f"{abbrev[item2]} -> {abbrev[item1]}",
        'Full Rule': f"{item2} -> {item1}",
        'Support': round(support_pair, 2),
        'Confidence': round(confidence_2_to_1, 2),
        'Calculation': f"{pair_count}/{frequent_1_itemsets[item2]} = {confidence_2_to_1:.2f}"
    })

df_rules_2 = pd.DataFrame(rules_2)
df_rules_2 = df_rules_2.sort_values('Confidence', ascending=False).reset_index(drop=True)

print("\nAssociation Rules from 2-Itemsets:")
print(df_rules_2[['Rule', 'Support', 'Confidence', 'Calculation']].to_string(index=False))


Association Rules from 2-Itemsets:
   Rule  Support  Confidence Calculation
 E -> M      0.5        0.71  5/7 = 0.71
 M -> E      0.5        0.71  5/7 = 0.71
Bu -> M      0.5        0.71  5/7 = 0.71
M -> Bu      0.5        0.71  5/7 = 0.71
 B -> E      0.4        0.67  4/6 = 0.67
B -> Bu      0.4        0.67  4/6 = 0.67
 E -> B      0.4        0.57  4/7 = 0.57
Bu -> B      0.4        0.57  4/7 = 0.57
Bu -> E      0.4        0.57  4/7 = 0.57
E -> Bu      0.4        0.57  4/7 = 0.57
 B -> M      0.3        0.50  3/6 = 0.50
 M -> B      0.3        0.43  3/7 = 0.43


## Step 5: Generate Association Rules from 3-Itemsets

### Understanding 3-Itemset Rules

From a 3-itemset {A, B, C}, we can generate **multiple types** of association rules:

### Type 1: Two Items -> One Item
Examples: {A, B} -> C, {A, C} -> B, {B, C} -> A

**Formula:**
```
Confidence({A, B} -> C) = Support(A, B, C) / Support(A, B)
                        = Count(A, B, C together) / Count(A, B together)
```

**Example:**
- Rule: **{Butter, Eggs} -> Milk**
- Count of {Butter, Eggs, Milk} together = 3 transactions
- Count of {Butter, Eggs} together = 4 transactions
- Confidence = 3/4 = 0.75 = **75%**
- **Interpretation:** When customers buy both Butter AND Eggs, they also buy Milk 75% of the time.

### Type 2: One Item -> Two Items
Examples: A -> {B, C}, B -> {A, C}, C -> {A, B}

**Formula:**
```
Confidence(A -> {B, C}) = Support(A, B, C) / Support(A)
                        = Count(A, B, C together) / Count(A)
```

**Example:**
- Rule: **Butter -> {Eggs, Milk}**
- Count of {Butter, Eggs, Milk} together = 3 transactions
- Count of Butter = 7 transactions
- Confidence = 3/7 = 0.43 = **43%**
- **Interpretation:** When customers buy Butter, they also buy BOTH Eggs AND Milk 43% of the time.

### Key Insight:
Type 1 rules ({A, B} -> C) typically have **higher confidence** than Type 2 rules (A -> {B, C}) because:
- Type 1: More specific antecedent (two items) = fewer transactions = higher ratio
- Type 2: General antecedent (one item) = more transactions = lower ratio

In [14]:
rules_3 = []

for triple, triple_count in frequent_3_itemsets.items():
    support_triple = triple_count / total_transactions
    
    # Generate all possible rules from this 3-itemset
    for i in range(len(triple)):
        # Single item consequent rules (e.g., {A, B} -> C)
        consequent = triple[i]
        antecedent = tuple(item for j, item in enumerate(triple) if j != i)
        
        # Get support of antecedent
        antecedent_sorted = tuple(sorted(antecedent))
        if antecedent_sorted in frequent_2_itemsets:
            antecedent_count = frequent_2_itemsets[antecedent_sorted]
            confidence = triple_count / antecedent_count
            
            ant_str = ', '.join([abbrev[item] for item in antecedent])
            cons_str = abbrev[consequent]
            
            rules_3.append({
                'Rule': f"{{{ant_str}}} -> {cons_str}",
                'Full Rule': f"{{{', '.join(antecedent)}}} -> {consequent}",
                'Support': round(support_triple, 2),
                'Confidence': round(confidence, 2),
                'Calculation': f"{triple_count}/{antecedent_count} = {confidence:.2f}"
            })
    
    # Two item consequent rules (e.g., A -> {B, C})
    for i in range(len(triple)):
        antecedent = triple[i]
        consequent = tuple(item for j, item in enumerate(triple) if j != i)
        
        # Get support of antecedent (single item)
        if antecedent in frequent_1_itemsets:
            antecedent_count = frequent_1_itemsets[antecedent]
            confidence = triple_count / antecedent_count
            
            ant_str = abbrev[antecedent]
            cons_str = ', '.join([abbrev[item] for item in consequent])
            
            rules_3.append({
                'Rule': f"{ant_str} -> {{{cons_str}}}",
                'Full Rule': f"{antecedent} -> {{{', '.join(consequent)}}}",
                'Support': round(support_triple, 2),
                'Confidence': round(confidence, 2),
                'Calculation': f"{triple_count}/{antecedent_count} = {confidence:.2f}"
            })

if len(rules_3) > 0:
    df_rules_3 = pd.DataFrame(rules_3)
    df_rules_3 = df_rules_3.sort_values('Confidence', ascending=False).reset_index(drop=True)
    
    print("\nAssociation Rules from 3-Itemsets:")
    print(df_rules_3[['Rule', 'Support', 'Confidence', 'Calculation']].to_string(index=False))
else:
    print("\nNo association rules from 3-itemsets (no frequent 3-itemsets found)")


Association Rules from 3-Itemsets:
        Rule  Support  Confidence Calculation
{Bu, E} -> M      0.3        0.75  3/4 = 0.75
{E, M} -> Bu      0.3        0.60  3/5 = 0.60
{Bu, M} -> E      0.3        0.60  3/5 = 0.60
Bu -> {E, M}      0.3        0.43  3/7 = 0.43
E -> {Bu, M}      0.3        0.43  3/7 = 0.43
M -> {Bu, E}      0.3        0.43  3/7 = 0.43


## Step 6: Contingency Table Analysis

### Understanding Contingency Tables

A **2x2 contingency table** helps us analyze the relationship between an antecedent and consequent in association rules.

We'll analyze the rule: **{Milk, Eggs} -> Butter**

The table shows four scenarios:
- **Has both** antecedent AND consequent
- **Has antecedent** but NOT consequent
- **Does NOT have antecedent** but HAS consequent
- **Has neither** antecedent NOR consequent

### Key Metrics:

**1. Support:**
```
Support({M, E} -> Bu) = Count({M, E, Bu}) / Total Transactions
```

**2. Confidence:**
```
Confidence({M, E} -> Bu) = Count({M, E, Bu}) / Count({M, E})
```

**3. Lift:**
```
Lift({M, E} -> Bu) = Confidence({M, E} -> Bu) / Support(Bu)
                    = [Count({M, E, Bu}) / Count({M, E})] / [Count(Bu) / Total]
```

**Lift Interpretation:**
- **Lift > 1**: Positive correlation (antecedent increases likelihood of consequent)
- **Lift = 1**: No correlation (independent)
- **Lift < 1**: Negative correlation (antecedent decreases likelihood of consequent)

### Observed vs Expected:

**Observed values:** Actual counts from the data

**Expected values:** What we would expect if items were independent (no association)
```
Expected = (Row Total × Column Total) / Grand Total
```

In [15]:
# Construct 2x2 Contingency Table for {Milk, Eggs} -> Butter
print("="*80)
print("CONTINGENCY TABLE ANALYSIS: {Milk, Eggs} -> Butter")
print("="*80)

# Count the four scenarios
has_ME_and_Bu = 0  # Has {Milk, Eggs} AND Butter
has_ME_not_Bu = 0  # Has {Milk, Eggs} but NOT Butter
not_ME_has_Bu = 0  # Does NOT have {Milk, Eggs} but HAS Butter
not_ME_not_Bu = 0  # Does NOT have {Milk, Eggs} AND does NOT have Butter

for transaction in transactions:
    has_milk = 'Milk' in transaction
    has_eggs = 'Eggs' in transaction
    has_butter = 'Butter' in transaction
    
    has_ME = has_milk and has_eggs
    
    if has_ME and has_butter:
        has_ME_and_Bu += 1
    elif has_ME and not has_butter:
        has_ME_not_Bu += 1
    elif not has_ME and has_butter:
        not_ME_has_Bu += 1
    else:
        not_ME_not_Bu += 1

# Calculate totals
total_has_ME = has_ME_and_Bu + has_ME_not_Bu
total_not_ME = not_ME_has_Bu + not_ME_not_Bu
total_has_Bu = has_ME_and_Bu + not_ME_has_Bu
total_not_Bu = has_ME_not_Bu + not_ME_not_Bu

print("\n1. OBSERVED CONTINGENCY TABLE:")
print("-" * 60)
print(f"                        Has Butter    No Butter    Row Total")
print(f"Has {{Milk, Eggs}}          {has_ME_and_Bu:^6}        {has_ME_not_Bu:^6}        {total_has_ME:^6}")
print(f"No {{Milk, Eggs}}           {not_ME_has_Bu:^6}        {not_ME_not_Bu:^6}        {total_not_ME:^6}")
print(f"Column Total            {total_has_Bu:^6}        {total_not_Bu:^6}        {total_transactions:^6}")

# Calculate expected values
expected_ME_Bu = (total_has_ME * total_has_Bu) / total_transactions
expected_ME_notBu = (total_has_ME * total_not_Bu) / total_transactions
expected_notME_Bu = (total_not_ME * total_has_Bu) / total_transactions
expected_notME_notBu = (total_not_ME * total_not_Bu) / total_transactions

print("\n2. EXPECTED CONTINGENCY TABLE (if independent):")
print("-" * 60)
print(f"                        Has Butter    No Butter    Row Total")
print(f"Has {{Milk, Eggs}}         {expected_ME_Bu:^7.1f}      {expected_ME_notBu:^7.1f}       {total_has_ME:^6}")
print(f"No {{Milk, Eggs}}          {expected_notME_Bu:^7.1f}      {expected_notME_notBu:^7.1f}       {total_not_ME:^6}")
print(f"Column Total            {total_has_Bu:^6}        {total_not_Bu:^6}        {total_transactions:^6}")

# Calculate metrics
support_ME = total_has_ME / total_transactions
support_Bu = total_has_Bu / total_transactions
support_ME_Bu = has_ME_and_Bu / total_transactions
confidence_ME_Bu = has_ME_and_Bu / total_has_ME if total_has_ME > 0 else 0
lift_ME_Bu = confidence_ME_Bu / support_Bu if support_Bu > 0 else 0

print("\n3. RULE METRICS: {{Milk, Eggs}} -> Butter")
print("-" * 60)
print(f"Support({{M, E}})        = {total_has_ME}/{total_transactions} = {support_ME:.2f}")
print(f"Support(Bu)            = {total_has_Bu}/{total_transactions} = {support_Bu:.2f}")
print(f"Support({{M, E, Bu}})    = {has_ME_and_Bu}/{total_transactions} = {support_ME_Bu:.2f}")
print(f"Confidence({{M,E}}->Bu) = {has_ME_and_Bu}/{total_has_ME} = {confidence_ME_Bu:.2f}")
print(f"Lift({{M,E}}->Bu)       = {confidence_ME_Bu:.2f}/{support_Bu:.2f} = {lift_ME_Bu:.2f}")

print("\n4. INTERPRETATION:")
print("-" * 60)
if lift_ME_Bu > 1:
    print(f"Lift = {lift_ME_Bu:.2f} > 1")
    print("POSITIVE CORRELATION: Buying {{Milk, Eggs}} INCREASES the likelihood of buying Butter")
    print(f"Customers who buy {{Milk, Eggs}} are {lift_ME_Bu:.2f}x more likely to buy Butter")
elif lift_ME_Bu < 1:
    print(f"Lift = {lift_ME_Bu:.2f} < 1")
    print("NEGATIVE CORRELATION: Buying {{Milk, Eggs}} DECREASES the likelihood of buying Butter")
else:
    print(f"Lift = {lift_ME_Bu:.2f} = 1")
    print("NO CORRELATION: {{Milk, Eggs}} and Butter are independent")

print("\n5. OBSERVED vs EXPECTED:")
print("-" * 60)
print(f"Observed {{M,E}} with Bu:  {has_ME_and_Bu}")
print(f"Expected {{M,E}} with Bu:  {expected_ME_Bu:.1f}")
if has_ME_and_Bu > expected_ME_Bu:
    print(f"Observed > Expected: There IS a positive association!")
elif has_ME_and_Bu < expected_ME_Bu:
    print(f"Observed < Expected: There IS a negative association!")
else:
    print(f"Observed = Expected: Items appear to be independent")

print("\n" + "="*80)

CONTINGENCY TABLE ANALYSIS: {Milk, Eggs} -> Butter

1. OBSERVED CONTINGENCY TABLE:
------------------------------------------------------------
                        Has Butter    No Butter    Row Total
Has {Milk, Eggs}            3             2             5   
No {Milk, Eggs}             4             1             5   
Column Total              7             3             10  

2. EXPECTED CONTINGENCY TABLE (if independent):
------------------------------------------------------------
                        Has Butter    No Butter    Row Total
Has {Milk, Eggs}           3.5          1.5           5   
No {Milk, Eggs}            3.5          1.5           5   
Column Total              7             3             10  

3. RULE METRICS: {{Milk, Eggs}} -> Butter
------------------------------------------------------------
Support({M, E})        = 5/10 = 0.50
Support(Bu)            = 7/10 = 0.70
Support({M, E, Bu})    = 3/10 = 0.30
Confidence({M,E}->Bu) = 3/5 = 0.60
Lift({M,E}->Bu) 

## Summary: Answer to Question

In [16]:
print("="*80)
print("SUMMARY FOR ASSIGNMENT")
print("="*80)
print(f"\nMinimum Support: {min_support} (count >= {min_count})\n")

print("\n1. FREQUENT 2-ITEMSETS:")
print("-" * 60)
print(df_2_itemsets[['Itemset', 'Count', 'Support']].to_string(index=False))

print("\n\n2. FREQUENT 3-ITEMSETS:")
print("-" * 60)
if len(frequent_3_itemsets) > 0:
    print(df_3_itemsets[['Itemset', 'Count', 'Support']].to_string(index=False))
else:
    print("No frequent 3-itemsets with support >= 0.3")

print("\n\n3. ASSOCIATION RULES WITH CONFIDENCE:")
print("-" * 60)
print("\nFrom 2-Itemsets:")
print(df_rules_2[['Rule', 'Confidence']].to_string(index=False))

if len(rules_3) > 0:
    print("\nFrom 3-Itemsets:")
    print(df_rules_3[['Rule', 'Confidence']].to_string(index=False))

print("\n" + "="*80)

SUMMARY FOR ASSIGNMENT

Minimum Support: 0.3 (count >= 3.0)


1. FREQUENT 2-ITEMSETS:
------------------------------------------------------------
Itemset  Count  Support
 {E, M}      5      0.5
{Bu, M}      5      0.5
 {B, E}      4      0.4
{B, Bu}      4      0.4
{Bu, E}      4      0.4
 {B, M}      3      0.3


2. FREQUENT 3-ITEMSETS:
------------------------------------------------------------
   Itemset  Count  Support
{Bu, E, M}      3      0.3


3. ASSOCIATION RULES WITH CONFIDENCE:
------------------------------------------------------------

From 2-Itemsets:
   Rule  Confidence
 E -> M        0.71
 M -> E        0.71
Bu -> M        0.71
M -> Bu        0.71
 B -> E        0.67
B -> Bu        0.67
 E -> B        0.57
Bu -> B        0.57
Bu -> E        0.57
E -> Bu        0.57
 B -> M        0.50
 M -> B        0.43

From 3-Itemsets:
        Rule  Confidence
{Bu, E} -> M        0.75
{E, M} -> Bu        0.60
{Bu, M} -> E        0.60
Bu -> {E, M}        0.43
E -> {Bu, M}        0.